In [1]:
import unittest

## Test Configuration

This is a conveniance function to get the username, password and client id from the OS level environment.

In [8]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

os.environ['ameritradeuser'] = 'test_user'
os.environ['ameritradepw'] = 'test_pw'
os.environ['ameritradeclientid'] = 'client_id'

class TestConfiguration(unittest.TestCase):
    def test_config_default(self):
        class_under_test = amc.AmeritradeRest()
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')  
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
    
    def test_config_parms(self):
        class_under_test = amc.AmeritradeRest('ameritradeuser', 'ameritradepw', 'ameritradeclientid')
        self.assertEqual(class_under_test.username, 'test_user')
        self.assertEqual(class_under_test.password, 'test_pw')
        self.assertEqual(class_under_test.client_id, 'client_id')
        self.assertEqual(class_under_test.consumer_key, 'client_id'+'@AMER.OAUTHAP')
        
TestConfiguration().test_config_default()
TestConfiguration().test_config_parms()

## Test Authorization

This requires you to have a developer account with Ameritrade (https://developer.tdameritrade.com).

In [10]:
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import os

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

class TestAuthorization(unittest.TestCase):
    def test_authorization(self):
        td_ameritrade = amc.AmeritradeRest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
        td_ameritrade.authenticate()
        self.assertGreater(len(td_ameritrade.authorization), 0)

TestAuthorization().test_authorization()

In [175]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import selenium.common.exceptions as selexcept
from pathlib import Path
import urllib
import requests
import time

browser = None
class AmeritradeRestTest: 
    
    def __init__(self, env_user_name='ameritradeuser', env_password='ameritradepw', env_client_id='ameritradeclientid'):
        self.user_data_dir = str(Path.home()) + r'\AppData\Local\Google\Chrome\User Data\tduser'
        self.callback_url = r'http://localhost'
        self.oauth_url = r'https://auth.tdameritrade.com/auth'
        self.oath_token_url = r'https://api.tdameritrade.com/v1/oauth2/token'
        self.configure_ameritrade(env_user_name, env_password, env_client_id)
    
    def configure_ameritrade(self, env_user_name='ameritradeuser', env_password='ameritradepw', env_client_id='ameritradeclientid'):
        """
        In order to keep developers from setting usernames and passwords in a file, the credentials will be stored in environment varialbes.
        """
        self.username = os.getenv(env_user_name)
        self.password = os.getenv(env_password)
        self.client_id = os.getenv(env_client_id)
        self.consumer_key = self.client_id + '@AMER.OAUTHAP'
    
    def authenticate(self):
        chrome_options = Options()
        chrome_options.add_argument('--user-data-dir='+self.user_data_dir)
        browser = webdriver.Chrome(options=chrome_options)
        try:
            # define the components of the url
            method = 'GET'
            payload = {
                        'response_type':'code',
                        'redirect_uri':self.callback_url,
                        'client_id':self.consumer_key
            }

            # build url
            built_url = requests.Request(method, self.oauth_url, params=payload).prepare().url

            # go to URL
            browser.get(built_url)
    
            #fill out form
            browser.find_element_by_id('username0').send_keys(self.username)
            browser.find_element_by_id('password1').send_keys(self.password)
            
            # click Login button
            browser.find_element_by_id('accept').click()
            
            # click allow on authorization screen
            browser.find_element_by_id('accept').click()

            # give it a second
            time.sleep(1)
            
            # At this point you get an error back since there is no localhost server. But the URL contains the authentication code
            new_url = urllib.parse.unquote(browser.current_url)

            # grab the URL and parse it for the auth code
            code = new_url.split('code=')[1]

            # Use the auth code to get an auth token
            #define the headers
            headers = {'Content-Type':'application/x-www-form-urlencoded'}

            # define payload
            payload = {
                'grant_type':'authorization_code',
                'access_type': 'offline',
                'code': code,
                'client_id': self.client_id,
                'redirect_uri': 'http://localhost'
            }

            # post the data to get a token
            authreply = requests.post(self.oath_token_url, headers=headers, data=payload)

            # convert json to dict
            self.authorization = authreply.json()
            return self.authorization
        except selexcept.NoSuchElementException as error:
            print(f'Error: {error}')
        except selexcept.WebDriverException as error:
            print(f'Error: {error}')
        finally:
            browser.close()

In [142]:
amct = AmeritradeRestTest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')

In [140]:
browser = webdriver.Chrome()

In [144]:
method = 'GET'
payload = {
            'response_type':'code',
            'redirect_uri':amct.callback_url,
            'client_id':amct.consumer_key
}

print(payload)

# build url
built_url = requests.Request(method, AmeritradeRestTest().oauth_url, params=payload).prepare().url
print(built_url)

# go to URL
browser.get(built_url)

{'response_type': 'code', 'redirect_uri': 'http://localhost', 'client_id': 'AQALSFXHGXYP4YSU8OBATCPZEYOSFTAQ@AMER.OAUTHAP'}
https://auth.tdameritrade.com/auth?response_type=code&redirect_uri=http%3A%2F%2Flocalhost&client_id=AQALSFXHGXYP4YSU8OBATCPZEYOSFTAQ%40AMER.OAUTHAP


In [145]:
browser.current_url

'https://auth.tdameritrade.com/auth?response_type=code&redirect_uri=http%3A%2F%2Flocalhost&client_id=AQALSFXHGXYP4YSU8OBATCPZEYOSFTAQ%40AMER.OAUTHAP'

In [174]:
amct = AmeritradeRestTest('maiotradeuser', 'maiotradepw', 'maiotradeclientid')
amct.authenticate()

{'access_token': 'SpR8EboYFYFu9eFc6JeYvQHR//73Mxu7UC+40bkbntpvys1MpVdJYVYhqS+LAPxvxbIJ1EGy288Kh/IF/seRgVcvNBOujIJ70LjKGjut1hfZTlRMclSNj0gXRtv6WXTFc3YlRCc6XFmdABIiNnGQRKvQUqKgZnDJFFG6xU4aLz3olNWwTb3ULnq7/PvvYGwBUZzQQU9KUJTrnwULXJ/1pU5XelE2ksq/umXgpo99l8THpxszXPM7Mr34XgXw0ooIf1rViLJ77WKDcJxMA1wSohIoKLfkynhyQ+LzUj/PaaV/LvNJ/VdVSNIeF6OcjJpu3Lny2GevCpg+1plL9hRqKL4jotWadol48/yiXOrUBUzYLIWuibHfMDwMYapGXCD78pN6BVT0dy1kEbiID4xPqyieQhYhJmQ9TwYK7AQ3nfHbd80EiTNhE5gI8RFnM10Ay+aoesQTlhTYRQ60NHyUFmx0oFSyTUiVltuYeurYh0Jd2oyFihVuk1g2LBssSxZ00GdCsV5oGTnNjroOowLRkRc6tVItene2S25zDXnkNJuXTeLB0qOuKFt/CpdPbmJ488qT+2whm2MAm2gbLMbXllnFm21f8UNCcMUEcWzAJCynXs3GwzZq/Bc+XqpzgXEt1Hkdq100MQuG4LYrgoVi/JHHvl59oscD1EV1LcSBoG8JdwTSEQHz8+Cf3RgRGcxgdUlo8MeK6Sq9M19j7n+VIQMYPV8vSj+1nsN2y+rJMSnBzudWXh/6pHZiQ1ao5Ls7ErN774tnxu8+YrhLZE7+IooBuS+TKnrRbthp2h7vs1Gkd3rfi5yCAVgnhuUSEc+PyZrePk2AUa0qEVO8yUYJlgjhcUWHs76n87eawXLfaSP2rnmUrlMXQrwILORgzhE36sykD6GtuUqYTgs4WfS/eA7ggcT2gxPZDc4lkJGaU+qsLwJlu86RUige2xfpYx87A6jL3FA+ES81bSXFlxI5Wg